In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader,TensorDataset
import torch.utils.data as data
import numpy as np
import scipy.io as sio
import math
import argparse
import random
import os

In [2]:
BATCH_SIZE = 128
EPISODE = 500000
TEST_EPISODE = 1000
LEARNING_RATE =1e-5
Nom = 1e-5
Weight_Deacy =1e-5
GPU = 0
Margin = 1

In [3]:
print("init dataset")
##################################参数##################################################################
dataroot = '../data'
dataset = 'APY_data'
image_embedding = 'res101'               #ResNet101层
class_embedding = 'att'         #属性表达 85-d
#######################################读取视觉特征###################################################################

matcontent = sio.loadmat(dataroot + "/" + dataset + "/" + image_embedding + ".mat")  #scipy loadmat
 
feature = matcontent['features'].T         #转置 30478x2048 每一行是一个完整的样本

label = matcontent['labels'].astype(int).squeeze() - 1   #matlab begin 1 ,numpy begin 0
########################################读取属性特征###########################################################

matcontent = sio.loadmat(dataroot + "/" + dataset + "/" + class_embedding + "_splits.mat")

    
# numpy array index starts from 0, matlab starts from 1
trainval_loc = matcontent['trainval_loc'].squeeze() - 1    #squeeze()去掉维度中的1 AxBx1 --->AxB

test_seen_loc = matcontent['test_seen_loc'].squeeze() - 1
test_unseen_loc = matcontent['test_unseen_loc'].squeeze() - 1

attribute = matcontent['att'].T    #转置 50x85 每行是整个属性向量

x = feature[trainval_loc]                      # train_features trainval里面是图片的编号 begin with 0 ，19832个
train_label = label[trainval_loc].astype(int)  # train_label  int类型没变 每个图片的lable 19832个
train_id = np.unique(train_label)

att = attribute[train_label]                   # train attributes 每个图片的属性 19832个

########################add negative pairs#######################
#x_negative = np.empty_like(x)
#x_negative_label = np.empty_like(train_label)
#print(x.shape[0])

#for i in range(x.shape[0]):
#    pick=np.random.choice(np.where(train_label[i]!=train_id)[0], replace=True)
#    x_negative[i] = x[pick]
#    x_negative_label[i] = train_label[pick]
    
x_test = feature[test_unseen_loc]                   # test_feature 5685个
test_label = label[test_unseen_loc].astype(int)     # test_label   5685个

x_test_seen = feature[test_seen_loc]                #test_seen_feature 4958个
test_label_seen = label[test_seen_loc].astype(int)  # test_seen_label  4958个
    
test_id = np.unique(test_label)                     # test_id  10个类 ，unique去重
att_pro = attribute[test_id]                        # test_attribute 每一类的属性向量 10x85

# train set
train_features = torch.from_numpy(x)   #np-->tensor
#train_fearures_negative = torch.from_numpy(x_negative)

sample_attributes=[]
train_label = torch.from_numpy(train_label).unsqueeze(1) #每张图片的属性转化 ，unsqueeze(1)就是插入到第一维度 AxB维-->Ax1xB
#train_negative_label = torch.from_numpy(x_negative_label).unsqueeze(1)
# attributes
all_attributes = np.array(attribute)  #所有50类属性转变为numpy数组???属性向量仍然用的numpy类型 没有转化为pytorch
#print(all_attributes)
    
#print('-'*50)
attributes = torch.from_numpy(attribute) 
#print(attribute)
# test set

test_features = torch.from_numpy(x_test)
#print(test_features.shape)

test_label = torch.from_numpy(test_label).unsqueeze(1)
#print(test_label.shape)

testclasses_id = np.array(test_id)
#print(testclasses_id.shape)

test_attributes = torch.from_numpy(att_pro).float()
#print(test_attributes.shape)

test_seen_features = torch.from_numpy(x_test_seen)
#print(test_seen_features.shape)

test_seen_label = torch.from_numpy(test_label_seen)

train_data = TensorDataset( train_label, train_features )
#train_data = TensorDataset(train_label, train_features, train_fearures_negative)

#################here need new code to make triplet data#####################
print('-'*100)

init dataset
----------------------------------------------------------------------------------------------------


In [4]:
from my_net_2 import AttributeNetwork
from my_net_2 import TripletNetwork

In [5]:
# init network
print("init networks")
attribute_network = AttributeNetwork(64,1024,1024,2048)  #85d属性 1024隐藏层 2048输出 85d到2048d
triplet_network = TripletNetwork(attribute_network)  #metric learning   
triplet_network.cuda(GPU)   
#attribute_network_optim = torch.optim.Adam(attribute_network.parameters(), lr=LEARNING_RATE, weight_decay=1e-5)
#优化器adam 学习率 正则1e-5

#attribute_network_scheduler = StepLR(attribute_network_optim, step_size=200000, gamma=0.5)
#学习率每200k步 乘0.5
#triplet_network_optim = torch.optim.Adam(triplet_network.parameters(), lr=LEARNING_RATE)
#
triplet_network_optim = torch.optim.Adam(triplet_network.parameters(), lr=LEARNING_RATE, 
                                         weight_decay=Weight_Deacy)
triplet_network_scheduler = StepLR(triplet_network_optim , step_size=200000 , gamma=0.5)
#
print('-'*100)

init networks
----------------------------------------------------------------------------------------------------


In [6]:
def compute_accuracy(test_features, test_label, test_id, test_attributes):
    
    test_data = TensorDataset(test_features, test_label)
    test_batch = 32
    test_loader = DataLoader(test_data, batch_size=test_batch, shuffle=False)
    total_rewards = 0

    sample_labels = test_id
    sample_attributes = test_attributes
    class_num = sample_attributes.shape[0]
    test_size = test_features.shape[0]

    print("class num:", class_num)

    for batch_features,batch_labels in test_loader:

        batch_size = batch_labels.shape[0]
        batch_features_ext = torch.from_numpy(batch_features.numpy().repeat(class_num, 0))
        batch_features_ext = Variable(batch_features_ext).cuda(GPU).float()  # 32*1024

        #print(batch_features_ext)

        sample_features = attribute_network(Variable(sample_attributes).cuda(GPU).float())
        sample_features_ext = sample_features.repeat(batch_size, 1)
        #print(sample_features_ext.shape)


        relations = F.pairwise_distance(batch_features_ext, sample_features_ext).view(-1, class_num)
        re_batch_labels = []
        for label in batch_labels.numpy():
            index = np.argwhere(sample_labels == label)
            re_batch_labels.append(index[0][0])
        re_batch_labels = torch.cuda.LongTensor(re_batch_labels)


        _, predict_labels = torch.min(relations.data, 1)
        #print(predict_labels)
        rewards = [1 if predict_labels[j] == re_batch_labels[j] else 0 for j in range(batch_size)]
        total_rewards += np.sum(rewards)
    test_accuracy = total_rewards/1.0/test_size
    return  test_accuracy

In [7]:
print("training...")
last_accuracy = 0.0
for episode in range(EPISODE):
   
    triplet_network_scheduler.step(episode)

    train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)

    batch_labels, batch_features = train_loader.__iter__().next()
    
    batch_features_negative = np.empty_like(batch_features.numpy())
    
    batch_attributes = torch.Tensor([all_attributes[i] for i in batch_labels.numpy()]).squeeze(1)
    
    batch_features = Variable(batch_features).cuda(GPU).float()  # 32*2048
    
    for i in range(batch_features_negative.shape[0]):
        pick=np.random.choice(np.where(batch_labels.numpy()!=train_id)[0], replace=True)
        batch_features_negative[i] = batch_features[pick]    
    
    batch_features_negative = torch.from_numpy(batch_features_negative)
    batch_features_negative = Variable(batch_features_negative).cuda(GPU).float()  # 32*2048
    
    #batch_attributes = attribute_network(Variable(batch_attributes).cuda(GPU))
    batch_attributes = Variable(batch_attributes).cuda(GPU)
    
    dista, distb, embedded_x, embedded_y, embedded_z = triplet_network(batch_attributes, batch_features, batch_features_negative)
    
    criterion = nn.MarginRankingLoss(margin = Margin).cuda(GPU)
    
    target = torch.FloatTensor(dista.size()).fill_(-1).cuda(GPU)
    
    
    loss_triplet= criterion(dista, distb, target)
    loss_embedd = embedded_x.norm(2) + embedded_y.norm(2) + embedded_z.norm(2)
    loss = loss_triplet + Nom * loss_embedd

    # update
    triplet_network.zero_grad()
    loss.backward()
    triplet_network_optim.step()
    
    if (episode+1)%1000 == 0:
        print("episode:", episode+1, "loss", loss.data)
    if (episode+1)%2000 == 0:
        print("Testing...")
        zsl_accuracy = compute_accuracy(test_features, test_label, test_id, test_attributes)
        gzsl_unseen_accuracy = compute_accuracy(test_features, test_label, np.arange(32), attributes)
        gzsl_seen_accuracy = compute_accuracy(test_seen_features, test_seen_label, np.arange(32), attributes)
        H = 2 * gzsl_seen_accuracy * gzsl_unseen_accuracy / (gzsl_unseen_accuracy + gzsl_seen_accuracy)
        print('zsl:', zsl_accuracy)
        print('gzsl: unseen=%.4f , seen=%.4f , h=%.4f' % (gzsl_unseen_accuracy , gzsl_seen_accuracy, H))
        print('_'*100)

training...
episode: 1000 loss tensor(2.2711, device='cuda:0')
episode: 2000 loss tensor(2.5374, device='cuda:0')
Testing...
class num: 12
class num: 32
class num: 32
zsl: 0.05502271580010096
gzsl: unseen=0.0550 , seen=0.0000 , h=0.0000
____________________________________________________________________________________________________
episode: 3000 loss tensor(2.5106, device='cuda:0')
episode: 4000 loss tensor(2.7050, device='cuda:0')
Testing...
class num: 12
class num: 32
class num: 32
zsl: 0.017541645633518427
gzsl: unseen=0.0000 , seen=0.0351 , h=0.0000
____________________________________________________________________________________________________
episode: 5000 loss tensor(2.3850, device='cuda:0')
episode: 6000 loss tensor(2.4804, device='cuda:0')
Testing...
class num: 12
class num: 32
class num: 32
zsl: 0.03634528016153458
gzsl: unseen=0.0191 , seen=0.0270 , h=0.0223
____________________________________________________________________________________________________
episode: 

episode: 51000 loss tensor(1.4873, device='cuda:0')
episode: 52000 loss tensor(2.0283, device='cuda:0')
Testing...
class num: 12
class num: 32
class num: 32
zsl: 0.09679454820797577
gzsl: unseen=0.0425 , seen=0.3972 , h=0.0768
____________________________________________________________________________________________________
episode: 53000 loss tensor(1.5743, device='cuda:0')
episode: 54000 loss tensor(1.6236, device='cuda:0')
Testing...
class num: 12
class num: 32
class num: 32
zsl: 0.09490156486622918
gzsl: unseen=0.0414 , seen=0.4032 , h=0.0751
____________________________________________________________________________________________________
episode: 55000 loss tensor(1.7086, device='cuda:0')
episode: 56000 loss tensor(1.5050, device='cuda:0')
Testing...
class num: 12
class num: 32
class num: 32
zsl: 0.09376577486118122
gzsl: unseen=0.0418 , seen=0.4100 , h=0.0758
____________________________________________________________________________________________________
episode: 57000 l

episode: 101000 loss tensor(1.4461, device='cuda:0')
episode: 102000 loss tensor(2.0331, device='cuda:0')
Testing...
class num: 12
class num: 32
class num: 32
zsl: 0.12695608278647147
gzsl: unseen=0.0512 , seen=0.4477 , h=0.0920
____________________________________________________________________________________________________
episode: 103000 loss tensor(1.3986, device='cuda:0')
episode: 104000 loss tensor(1.6085, device='cuda:0')
Testing...
class num: 12
class num: 32
class num: 32
zsl: 0.13162544169611307
gzsl: unseen=0.0540 , seen=0.4464 , h=0.0964
____________________________________________________________________________________________________
episode: 105000 loss tensor(1.4973, device='cuda:0')
episode: 106000 loss tensor(1.7333, device='cuda:0')
Testing...
class num: 12
class num: 32
class num: 32
zsl: 0.12809187279151943
gzsl: unseen=0.0536 , seen=0.4484 , h=0.0958
____________________________________________________________________________________________________
episode: 1

episode: 151000 loss tensor(1.2398, device='cuda:0')
episode: 152000 loss tensor(1.5515, device='cuda:0')
Testing...
class num: 12
class num: 32
class num: 32
zsl: 0.15030287733467945
gzsl: unseen=0.0876 , seen=0.4788 , h=0.1481
____________________________________________________________________________________________________
episode: 153000 loss tensor(1.6514, device='cuda:0')
episode: 154000 loss tensor(1.5124, device='cuda:0')
Testing...
class num: 12
class num: 32
class num: 32
zsl: 0.14979808177688037
gzsl: unseen=0.0864 , seen=0.4761 , h=0.1463
____________________________________________________________________________________________________
episode: 155000 loss tensor(1.5175, device='cuda:0')
episode: 156000 loss tensor(1.3229, device='cuda:0')
Testing...
class num: 12
class num: 32
class num: 32
zsl: 0.14967188288743058
gzsl: unseen=0.0861 , seen=0.4720 , h=0.1456
____________________________________________________________________________________________________
episode: 1

episode: 201000 loss tensor(1.8491, device='cuda:0')
episode: 202000 loss tensor(1.8322, device='cuda:0')
Testing...
class num: 12
class num: 32
class num: 32
zsl: 0.1472741039878849
gzsl: unseen=0.0896 , seen=0.4552 , h=0.1497
____________________________________________________________________________________________________
episode: 203000 loss tensor(1.3953, device='cuda:0')
episode: 204000 loss tensor(1.0879, device='cuda:0')
Testing...
class num: 12
class num: 32
class num: 32
zsl: 0.1491670873296315
gzsl: unseen=0.0909 , seen=0.4531 , h=0.1514
____________________________________________________________________________________________________
episode: 205000 loss tensor(1.7443, device='cuda:0')
episode: 206000 loss tensor(1.3401, device='cuda:0')
Testing...
class num: 12
class num: 32
class num: 32
zsl: 0.14853609288238265
gzsl: unseen=0.0900 , seen=0.4518 , h=0.1501
____________________________________________________________________________________________________
episode: 207

episode: 251000 loss tensor(1.0807, device='cuda:0')
episode: 252000 loss tensor(1.2988, device='cuda:0')
Testing...
class num: 12
class num: 32
class num: 32
zsl: 0.1514386673397274
gzsl: unseen=0.0959 , seen=0.4477 , h=0.1580
____________________________________________________________________________________________________
episode: 253000 loss tensor(1.7758, device='cuda:0')
episode: 254000 loss tensor(1.4221, device='cuda:0')
Testing...
class num: 12
class num: 32
class num: 32
zsl: 0.1514386673397274
gzsl: unseen=0.0977 , seen=0.4491 , h=0.1605
____________________________________________________________________________________________________
episode: 255000 loss tensor(1.4937, device='cuda:0')
episode: 256000 loss tensor(1.3842, device='cuda:0')
Testing...
class num: 12
class num: 32
class num: 32
zsl: 0.15118626956082787
gzsl: unseen=0.0982 , seen=0.4484 , h=0.1611
____________________________________________________________________________________________________
episode: 257

episode: 301000 loss tensor(1.7134, device='cuda:0')
episode: 302000 loss tensor(0.9526, device='cuda:0')
Testing...
class num: 12
class num: 32
class num: 32
zsl: 0.16014639071176173
gzsl: unseen=0.0989 , seen=0.4390 , h=0.1615
____________________________________________________________________________________________________
episode: 303000 loss tensor(1.5628, device='cuda:0')
episode: 304000 loss tensor(1.2267, device='cuda:0')
Testing...
class num: 12
class num: 32
class num: 32
zsl: 0.16140837960625948
gzsl: unseen=0.1016 , seen=0.4410 , h=0.1651
____________________________________________________________________________________________________
episode: 305000 loss tensor(1.3543, device='cuda:0')
episode: 306000 loss tensor(1.4906, device='cuda:0')
Testing...
class num: 12
class num: 32
class num: 32
zsl: 0.1625441696113074
gzsl: unseen=0.1018 , seen=0.4403 , h=0.1654
____________________________________________________________________________________________________
episode: 30

episode: 351000 loss tensor(1.3663, device='cuda:0')
episode: 352000 loss tensor(1.5393, device='cuda:0')
Testing...
class num: 12
class num: 32
class num: 32
zsl: 0.17579505300353357
gzsl: unseen=0.1097 , seen=0.4376 , h=0.1754
____________________________________________________________________________________________________
episode: 353000 loss tensor(1.4502, device='cuda:0')
episode: 354000 loss tensor(1.9587, device='cuda:0')
Testing...
class num: 12
class num: 32
class num: 32
zsl: 0.17592125189298335
gzsl: unseen=0.1098 , seen=0.4363 , h=0.1754
____________________________________________________________________________________________________
episode: 355000 loss tensor(0.8437, device='cuda:0')
episode: 356000 loss tensor(1.2750, device='cuda:0')
Testing...
class num: 12
class num: 32
class num: 32
zsl: 0.1765522463402322
gzsl: unseen=0.1109 , seen=0.4356 , h=0.1768
____________________________________________________________________________________________________
episode: 35

episode: 401000 loss tensor(1.4709, device='cuda:0')
episode: 402000 loss tensor(1.0469, device='cuda:0')
Testing...
class num: 12
class num: 32
class num: 32
zsl: 0.198889449772842
gzsl: unseen=0.1172 , seen=0.4322 , h=0.1844
____________________________________________________________________________________________________
episode: 403000 loss tensor(1.4580, device='cuda:0')
episode: 404000 loss tensor(1.2019, device='cuda:0')
Testing...
class num: 12
class num: 32
class num: 32
zsl: 0.19989904088844018
gzsl: unseen=0.1161 , seen=0.4329 , h=0.1831
____________________________________________________________________________________________________
episode: 405000 loss tensor(1.4789, device='cuda:0')
episode: 406000 loss tensor(1.0310, device='cuda:0')
Testing...
class num: 12
class num: 32
class num: 32
zsl: 0.20128722867238769
gzsl: unseen=0.1164 , seen=0.4322 , h=0.1834
____________________________________________________________________________________________________
episode: 407

episode: 451000 loss tensor(1.0398, device='cuda:0')
episode: 452000 loss tensor(1.8734, device='cuda:0')
Testing...
class num: 12
class num: 32
class num: 32
zsl: 0.2147905098435134
gzsl: unseen=0.1210 , seen=0.4343 , h=0.1893
____________________________________________________________________________________________________
episode: 453000 loss tensor(1.3898, device='cuda:0')
episode: 454000 loss tensor(1.7311, device='cuda:0')
Testing...
class num: 12
class num: 32
class num: 32
zsl: 0.21529530540131248
gzsl: unseen=0.1209 , seen=0.4322 , h=0.1889
____________________________________________________________________________________________________
episode: 455000 loss tensor(1.5336, device='cuda:0')
episode: 456000 loss tensor(1.4820, device='cuda:0')
Testing...
class num: 12
class num: 32
class num: 32
zsl: 0.21870267541645633
gzsl: unseen=0.1218 , seen=0.4322 , h=0.1900
____________________________________________________________________________________________________
episode: 45